In [1]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    count = 0
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for page to load
        time.sleep(random.uniform(1.5, 2.5))

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            count += 1
            if count == 5:
                break
        else:
            last_height = new_height
            count = 0

        # Simulate human-like scrolling by scrolling in smaller increments
        #for i in range(3):
         #   driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
          #  time.sleep(random.uniform(0.5, 1))

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import random
url = 'https://ta.quora.com/topic/%E0%AE%87%E0%AE%AF%E0%AE%B1%E0%AF%8D%E0%AE%AA%E0%AE%BF%E0%AE%AF%E0%AE%B2%E0%AF%8D'

# Load the web page
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()
driver.get(url)
scroll_to_bottom(driver)
print('scrolling completed')
# Find all clickable "மேலும் காண்க" elements
wait = WebDriverWait(driver, 10)
elements = wait.until(ec.presence_of_all_elements_located((By.XPATH, "//*[contains(text(), 'மேலும் காண்க')]")))
clickable_elements = []
for element in elements:
    if element.is_displayed() and element.is_enabled():
        clickable_elements.append(element)

# Click all the clickable "மேலும் காண்க" elements
for element in clickable_elements:
    driver.execute_script("arguments[0].click();", element)
print('clicking complete')

scrolling completed
clicking complete


In [3]:
html = driver.page_source


In [18]:
from bs4 import BeautifulSoup
import json
soup = BeautifulSoup(html, 'html.parser')

soup = BeautifulSoup(html, 'html.parser')

# Find all elements with class 'qu-mb--tiny'
small_classes = soup.find_all(class_='qu-mb--tiny')
x = 0
# Loop over the small_classes and find the adjacent 'spacing_log_answer_content' class
for small_class in small_classes:
    adjacent_class = small_class.find_next(class_='spacing_log_answer_content')
    if adjacent_class:     
        mydict = {}
        question = small_class.text.strip()
        answer = adjacent_class.text.strip()
        if len(question)>30:
            x = x+1
            mydict["question"] = question
            mydict["answer"] = answer
            # Open the JSON file in append mode
            with open("qna_quora.txt", "a",encoding='utf-8') as f:
                f.write(mydict)          

#finding next sibling
from bs4 import BeautifulSoup
# Create a BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')
question_elements = soup.find_all(class_='qu-mb--tiny')
# Find all 'qu-mb--tiny' elements and iterate over each of them
count = 0
for question in question_elements:
    # Find the next sibling element with class 'spacing_log_answer_content'
    #answer = question.find_next_sibling(class_='spacing_log_answer_content')  
    # Extract the text from both elements
    if question:
        #question_text = question.text(strip=True)
        #answer_text = answer.text(strip=True)
        # Print the question and answer
        count+=1
        print(count)
        print(question.get_text().strip())
        #print('Answer:', answer_text)
        print('------------------------')
